# Goodreads Data Extraction And Load Into Database

In this notebook we will do the below steps:
1. We will fetch the book details from goodreads using an API
2. We will create a SQLite Database connection and create tables
3. We will load the search results into the tables created above
4. We will close the Databse Connection.

## Step 1: Data Extraction

### 1.1 Import Libraries

In [22]:
import pickle
# Import goodreads api
from goodreads import client
from goodreads.client import GoodreadsClient
from goodreads.book import GoodreadsBook
from goodreads.author import GoodreadsAuthor
from goodreads.shelf import GoodreadsShelf
from goodreads.review import GoodreadsReview

# Database
from sqlalchemy import create_engine, inspect
from sqlalchemy.exc import IntegrityError
from sqlalchemy import Table, Column, Integer,Float, String, MetaData,ForeignKey,insert
import sqlite3
from sqlite3 import Error

### 1.2 Function to Authenticate GoodReads Access

To access the GoodReads Data, we can use the GoodReads API. But to use the GoodReads API, we need to register for a developer key. The key can be registered on https://www.goodreads.com/api/keys.
<br>The credentials are secret information and thus can be stored in a pkl file and loaded when required.

In [23]:
def auth_access():
    """A Function to authenticate GoodReads Data Access"""
    ## Get the credentials for goodreads
    goodreads_credentials = open("goodreads_credentials.pkl","rb")
    goodreads = pickle.load(goodreads_credentials)
    
    ## Pass the Key and Secret Code to create a client instance
    key = goodreads['key']
    secret = goodreads['secret']
    gc = client.GoodreadsClient(key, secret)
    return(gc)

### 1.3 Function to search books from goodreads by genre

1. Pass a genre as a parameter
2. Search Books for that genre using the Goodreads API method **search_books**
3. Insert the search results into Database table **Books** using the function **insert_book_data**.

In [24]:
def search_books(search_list):
    """A Function to search books from goodreads by a genre"""
    # Authenticate access
    gc=auth_access()
    # Instialize a Book list
    books = []
    for genre in search_list:
        print("search_books :Start searching books for genre= ", genre)
        for i in range(101):
            try:
                # Call goodreads API to search for the genre passed as parameter
                books.extend(gc.search_books(q=genre,page=i+1,search_field='genre'))  
            except TypeError as exc:
                print("In TypeError Exception in search_books. Error: ",str(exc))
                pass
            except Exception as inst:
                # handle all other exceptions
                print("Unexpected Exception:")
                pass
        print("No. of Books found for the genre= ", len(books))
    print("Total No. of Books found for the searched genre list= ", len(books))
    insert_book_data(books)

## Step 2: Database Design

**SQLite** is a server-less *Relational Database Management System*.It is amazing how SQLite stores the entire database (definitions, tables, indices, and the data itself) as a single file on a host machine. 
<br><br>
SQLite read operations can be multitasked. 
<br >But SQLite write operations can only be performed sequentially since the entire database file is locked during writing.

### 2.1  Create a SQLite Database Engine and Connect

In [25]:
engine = create_engine('sqlite:///goodreads_db.db.sqlite')
conn = engine.connect()
trans = conn.begin()
meta = MetaData(engine)
meta.drop_all(engine)

### 2.2  Prepare Tables Creation Script

We are extracting both the **Books** details and the **Author** information grom goodreads for a particular genre.
<br> Thus we need to create 2 tables as below:
1. Books - To store the book details where book_id is the **Primary key**
2. Authors - To store the author details where book_id from Books table is the **Foreign key**

In [26]:
# create books table
books = Table(
   'books', meta, 
   Column('book_id', Integer, primary_key = True), 
   Column('isbn', String), 
   Column('title', String), 
   Column('total_pages', Float), 
   Column('average_rating', Float), 
   Column('ratings_count', Integer), 
   Column('reviews_count', Integer), 
   Column('publication_date', String), 
   Column('publisher', String), 
   Column('popular_shelves', String), 
   Column('book_description', String)
    )
# create authors table
authors = Table(
   'authors', meta, 
   Column('book_id', Integer, ForeignKey('books.book_id')), 
   Column('author_id', Integer), 
   Column('author_name', String), 
   Column('birth_on', String), 
   Column('death_on', String), 
   Column('fans_count', Integer), 
   Column('gender', String), 
   Column('hometown', String), 
   Column('works_count', Integer)
    )

### 2.3  Create Tables

In [27]:
meta.create_all(engine)

### 2.4 Check Tables in Database

In [28]:
table_names = engine.table_names()
print(table_names)

['authors', 'books']


### 2.5 Create Function to insert records into Books Table

In [29]:
def insert_book_data(input):
    """A Function to insert records into books table"""
    print("insert_book_data :Start inserting data into books table")
    conn.execute(books.delete())
    ins = books.insert()
    counter = 0
    authors =[]
    for book in input:
        try:
            conn.execute(ins.values(book_id=book.gid,
                          isbn=book.isbn, 
                          title=book.title,
                          total_pages=book.num_pages,
                          average_rating=book.average_rating,
                          ratings_count=book.ratings_count,
                          reviews_count=book.text_reviews_count,
                          publication_date=','.join(book.publication_date),
                          publisher=book.publisher,
                          popular_shelves=','.join(str(v) for v in book.popular_shelves if book.popular_shelves),
                          book_description=book.description))
            counter += 1
            authors.extend([[book.gid,book.authors[0]]])
            
        except ValueError as e:
            print("In insert_book_data, ValueError Exception. Error: ",str(e))
            pass
        except TypeError as exc:
            #print("In insert_book_data, TypeError Exception. Error: ",str(exc))
            try:
                    #if book.gid not in books_inserted:
                    conn.execute(ins.values(book_id=book.gid,
                                  isbn=book.isbn, 
                                  title=book.title,
                                  total_pages=book.num_pages,
                                  average_rating=book.average_rating,
                                  ratings_count=book.ratings_count,
                                  reviews_count=book.text_reviews_count,
                                  publisher=book.publisher,
                                  book_description=book.description))
                    counter += 1
                    authors.extend([[book.gid,book.authors[0]]])
            except Exception as e:
                print("Exception: ", str(e))
                pass
        except IntegrityError as e:
            print("In insert_book_data, IntegrityError Exception for book_id: ",book.gid)
            pass
    print("Records inserted into Books table= ", counter)
    insert_author_data(authors)

### 2.6 Create Function to insert records into Authors Table

In [30]:
def insert_author_data(au_input):
    """A Function to insert records into authors table"""
    print("insert_author_data :Start inserting data into authors table")
    conn.execute(authors.delete())
    au_ins  = authors.insert()
    cnt = 0
    gc=auth_access()
    for i in range(len(au_input)):
        try:
            book_id = au_input[i][0]
            name=au_input[i][1]
            author = gc.find_author(name)
            conn.execute(au_ins.values(book_id=book_id,
                  author_id=author.gid, 
                  author_name=author.name,
                  birth_on=author.born_at,
                  death_on=author.died_at,
                  fans_count=author.fans_count()['#text'],
                  gender=author.gender,
                  hometown=author.hometown,
                  works_count=author.works_count))
            cnt += 1
        except ValueError as e:
            print("In insert_author_data, ValueError Exception. Error: ",str(e))
            pass
        except TypeError as exc:
            print("In insert_author_data, TypeError Exception. Error: ",str(exc))
            pass
        except Exception as e:
                print("Authors Exception: ", str(e))
                pass
    print("Records inserted into authors table= ", cnt)

## Step3: Extract and  Populate Tables

In this project, we are extracting books details for the **"Science Fiction"** Genre. We will use the below tags to get a recpectable amount of data:
1. science fiction
2. science-fiction-fantasy
3. science-fiction-romance
4. Apocalyptic
5. Space
6. Dystopia
7. Aliens
8. Fantasy

In [31]:
search_list = ['science fiction','science-fiction-fantasy','science-fiction-romance','Apocalyptic','Space','Dystopia','Aliens','Fantasy']
search_books(search_list)
trans.commit()

search_books :Start searching books for genre=  science fiction
No. of Books found for the genre=  2020
search_books :Start searching books for genre=  science-fiction-fantasy
No. of Books found for the genre=  4040
search_books :Start searching books for genre=  science-fiction-romance
Unexpected Exception:
Unexpected Exception:
No. of Books found for the genre=  6020
search_books :Start searching books for genre=  Apocalyptic
No. of Books found for the genre=  8040
search_books :Start searching books for genre=  Space
No. of Books found for the genre=  10060
search_books :Start searching books for genre=  Dystopia
Unexpected Exception:
Unexpected Exception:
No. of Books found for the genre=  12040
search_books :Start searching books for genre=  Aliens
No. of Books found for the genre=  14060
search_books :Start searching books for genre=  Fantasy
No. of Books found for the genre=  16080
Total No. of Books found for the searched genre list=  16080
insert_book_data :Start inserting dat

In insert_book_data, IntegrityError Exception for book_id:  13651
In insert_book_data, IntegrityError Exception for book_id:  6080337
In insert_book_data, IntegrityError Exception for book_id:  29981
Exception:  (sqlite3.IntegrityError) UNIQUE constraint failed: books.book_id [SQL: 'INSERT INTO books (book_id, isbn, title, total_pages, average_rating, ratings_count, reviews_count, publisher, book_description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: ('18130', None, 'A Wind in the Door (Time Quintet, #2)', 203.0, 4.04, '96154', '2295', 'Dell Laurel-Leaf', "<b>Every time a star goes out, another Echthros has won a battle.</b><br /><br />Just before Meg Murry's little brother, Charles Wallace, falls death ... (287 characters truncated) ... s Wallace to attempt to defeat the Echthroi—those who hate—and restore brilliant harmony and joy to the rhythm of creation, the song of the universe.")] (Background on this error at: http://sqlalche.me/e/gkpj)
In insert_book_data, IntegrityErro

In insert_book_data, IntegrityError Exception for book_id:  9593
In insert_book_data, IntegrityError Exception for book_id:  357
In insert_book_data, IntegrityError Exception for book_id:  433567
In insert_book_data, IntegrityError Exception for book_id:  9534
In insert_book_data, IntegrityError Exception for book_id:  223380
In insert_book_data, IntegrityError Exception for book_id:  30013
In insert_book_data, IntegrityError Exception for book_id:  76683
In insert_book_data, IntegrityError Exception for book_id:  77711
In insert_book_data, IntegrityError Exception for book_id:  40604754
In insert_book_data, IntegrityError Exception for book_id:  16433
In insert_book_data, IntegrityError Exception for book_id:  100365
In insert_book_data, IntegrityError Exception for book_id:  7661
In insert_book_data, IntegrityError Exception for book_id:  59924
In insert_book_data, IntegrityError Exception for book_id:  4703581
Exception:  (sqlite3.IntegrityError) UNIQUE constraint failed: books.book

In insert_book_data, IntegrityError Exception for book_id:  46756
In insert_book_data, IntegrityError Exception for book_id:  7576115
In insert_book_data, IntegrityError Exception for book_id:  47994
In insert_book_data, IntegrityError Exception for book_id:  105
In insert_book_data, IntegrityError Exception for book_id:  70539
In insert_book_data, IntegrityError Exception for book_id:  823
In insert_book_data, IntegrityError Exception for book_id:  346087
In insert_book_data, IntegrityError Exception for book_id:  826845
Exception:  (sqlite3.IntegrityError) UNIQUE constraint failed: books.book_id [SQL: 'INSERT INTO books (book_id, isbn, title, total_pages, average_rating, ratings_count, reviews_count, publisher, book_description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: ('216442', '0553560719', 'Dark Force Rising (Star Wars: The Thrawn Trilogy, #2)', 439.0, 4.15, '42012', '846', 'Del Rey Books', "The dying Empire's most cunning and ruthless warlord—Grand Admiral Thrawn—has ta

In insert_book_data, IntegrityError Exception for book_id:  76658
In insert_book_data, IntegrityError Exception for book_id:  22323
In insert_book_data, IntegrityError Exception for book_id:  28547
In insert_book_data, IntegrityError Exception for book_id:  48484
In insert_book_data, IntegrityError Exception for book_id:  7507908
In insert_book_data, IntegrityError Exception for book_id:  28547
In insert_book_data, IntegrityError Exception for book_id:  22341276
In insert_book_data, IntegrityError Exception for book_id:  429138
In insert_book_data, IntegrityError Exception for book_id:  21793182
In insert_book_data, IntegrityError Exception for book_id:  8477057
In insert_book_data, IntegrityError Exception for book_id:  18616975
In insert_book_data, IntegrityError Exception for book_id:  17571466
In insert_book_data, IntegrityError Exception for book_id:  295086
In insert_book_data, IntegrityError Exception for book_id:  7849341
In insert_book_data, IntegrityError Exception for book_i

...

In insert_book_data, IntegrityError Exception for book_id:  17404295
In insert_book_data, IntegrityError Exception for book_id:  9859436
In insert_book_data, IntegrityError Exception for book_id:  2754510
In insert_book_data, IntegrityError Exception for book_id:  6933876
In insert_book_data, IntegrityError Exception for book_id:  34534
In insert_book_data, IntegrityError Exception for book_id:  16136599
In insert_book_data, IntegrityError Exception for book_id:  10790516
In insert_book_data, IntegrityError Exception for book_id:  29772863
In insert_book_data, IntegrityError Exception for book_id:  12637490
In insert_book_data, IntegrityError Exception for book_id:  12119529
In insert_book_data, IntegrityError Exception for book_id:  13323842
Exception:  (sqlite3.IntegrityError) UNIQUE constraint failed: books.book_id [SQL: 'INSERT INTO books (book_id, isbn, title, total_pages, average_rating, ratings_count, reviews_count, publisher, book_description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)

In insert_book_data, IntegrityError Exception for book_id:  395851
In insert_book_data, IntegrityError Exception for book_id:  18138189
In insert_book_data, IntegrityError Exception for book_id:  36502054
In insert_book_data, IntegrityError Exception for book_id:  530848
In insert_book_data, IntegrityError Exception for book_id:  8965
In insert_book_data, IntegrityError Exception for book_id:  77160
In insert_book_data, IntegrityError Exception for book_id:  19090384
In insert_book_data, IntegrityError Exception for book_id:  15703770
In insert_book_data, IntegrityError Exception for book_id:  6723348
In insert_book_data, IntegrityError Exception for book_id:  11556960
In insert_book_data, IntegrityError Exception for book_id:  15858
In insert_book_data, IntegrityError Exception for book_id:  6344423
In insert_book_data, IntegrityError Exception for book_id:  715791
In insert_book_data, IntegrityError Exception for book_id:  20168816
In insert_book_data, IntegrityError Exception for bo

In insert_book_data, IntegrityError Exception for book_id:  68497
In insert_book_data, IntegrityError Exception for book_id:  17563080
In insert_book_data, IntegrityError Exception for book_id:  140097
In insert_book_data, IntegrityError Exception for book_id:  60211
In insert_book_data, IntegrityError Exception for book_id:  7519472
In insert_book_data, IntegrityError Exception for book_id:  17901125
In insert_book_data, IntegrityError Exception for book_id:  12027429
In insert_book_data, IntegrityError Exception for book_id:  11882171
In insert_book_data, IntegrityError Exception for book_id:  30226
Exception:  (sqlite3.IntegrityError) UNIQUE constraint failed: books.book_id [SQL: 'INSERT INTO books (book_id, isbn, title, total_pages, average_rating, ratings_count, reviews_count, publisher, book_description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: ('16130758', None, 'Shadow Study (Soulfinders #1; Study, #4)', 384.0, 4.08, '9688', '955', 'Mira', '<i>Librarian\'s Note: This i

In insert_book_data, IntegrityError Exception for book_id:  18602406
In insert_book_data, IntegrityError Exception for book_id:  28512486
Exception:  (sqlite3.IntegrityError) UNIQUE constraint failed: books.book_id [SQL: 'INSERT INTO books (book_id, isbn, title, total_pages, average_rating, ratings_count, reviews_count, publisher, book_description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: ('841628', '0330316117', "The Hitchhiker's Guide to the Galaxy: A Trilogy in Four Parts", 590.0, 4.51, '30702', '375', 'Pan Books', '<i>Charting the whole of Arthur Dent\'s odyssey through space are:</i><br /><br />THE HITCHHIKER\'S GUIDE TO THE GALAXY.<br />One Thursday lunchtime  ... (1387 characters truncated) ... people will find oddly familiar. They go in search of God\'s Final Message to His Creation and, in a dramatic break with tradition, actually find it.')] (Background on this error at: http://sqlalche.me/e/gkpj)
In insert_book_data, IntegrityError Exception for book_id:  8535273
In

In insert_book_data, IntegrityError Exception for book_id:  8447255
In insert_book_data, IntegrityError Exception for book_id:  13539191
In insert_book_data, IntegrityError Exception for book_id:  2204464
In insert_book_data, IntegrityError Exception for book_id:  13138736
In insert_book_data, IntegrityError Exception for book_id:  43917
In insert_book_data, IntegrityError Exception for book_id:  13246736
In insert_book_data, IntegrityError Exception for book_id:  30123388
In insert_book_data, IntegrityError Exception for book_id:  23346358
Exception:  (sqlite3.IntegrityError) UNIQUE constraint failed: books.book_id [SQL: 'INSERT INTO books (book_id, isbn, title, total_pages, average_rating, ratings_count, reviews_count, publisher, book_description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: ('1051620', '1841496049', 'The Innocent Mage (Kingmaker, Kingbreaker, #1)', 642.0, 3.65, '16605', '746', 'Orbit', "Enter the kingdom of Lur, where to use magic unlawfully means death. The Do

## Step 4: Close Database Connection

In [32]:
conn.close()